In [1]:
import pandas as pd

In [6]:
metric_list = ['mae', 'mse', 'rmse', 'smape', 'mase']
all_metrics = [1, 2, 3, 4, 5]

df = pd.DataFrame([all_metrics], columns=metric_list)
df

,mae,mse,rmse,smape,mase
0,1,2,3,4,5
